# Custom Pipeline for Running YOLOv8 Model for Image Classification

- The unified framework from YOLOv8 (You Only Look Once version 8) can handle both object detection and classification, making it versatile for tasks requiring both capabilities. Also, this model incorporates modern deep learning techniques and optimizations, providing a good balance of speed and accuracy. Finally, it has a mature API for fine-tuning our own dataset.

- Seealso [yuting public repo](https://github.com/ytsimon2004/deep_imgcls)
    - [pipleine](https://github.com/ytsimon2004/deep_imgcls/blob/main/src/imgcls/classification/yolov8/pipeline.py)
    - [notebook example](https://github.com/ytsimon2004/deep_imgcls/tree/main/notebook)


- For more information on training modes, visit [Ultralytics Documentation](https://docs.ultralytics.com/modes/train/).
    - Avalible model type: `yolov8n`, `yolov8s`, `yolov8m`, `yolov8l`, `yolov8x`


## Follow the data folder structure as outlined below**:
```
SOURCE_ROOT_DIR (in the kaggle demo below, use '/kaggle/working')
    │
    ├── dataset.yml (1)
    ├── runs/ (2)
    │   └── detect
    │         ├── predict*
    │         │    ├── labels/
    │         │    ├── test_*.png
    │         │    └── test_set.csv (5)
    │         ├── train*
    │         │    ├── <yolo outputs>
    │         │    └── weights/ (6)
    │         └── *yolov8s.pt
    │
    ├── test/ (3)
    │   ├── img/
    │   ├── img_png/
    │   └── test_set.csv
    │
    └── train (4)
        ├── img/
        ├── img_png/
        ├── seg/
        ├── seg_png/
        └── train_set.csv
        
```
    (1) config yaml file for the custom path info/image labels
    
    (2) directory for model/train/evaluation output files
    
    (3) directory for test dataset
    
    (4) directory for train dataset

    (5) output results of classification
    
    (6) store the fine-tuned model weights


## Dependencies installation

In [ ]:
!pip install ultralytics==8.2.2

In [ ]:
from pathlib import Path
from pprint import pprint
from typing import final, Final, Literal, NamedTuple
from typing_extensions import TypeAlias

import os
import cv2
import numpy as np
import polars as pl
import torch
import yaml
import shutil
from PIL import Image
from matplotlib import pyplot as plt, patches
from skimage.measure import regionprops, label
from tqdm import tqdm
from ultralytics import YOLO
from colorama import Fore, Style
from datetime import datetime

In [ ]:
# handle wandb API checking bug
os.environ['WANDB_MODE'] = 'disabled'
os.environ["WANDB_DISABLED"] = "true"

## Helper Class/Function
- copy from personal repo for easilier usage in this notebook

In [ ]:
ClassInt: TypeAlias = int  # image class
ClassName: TypeAlias = str  # image class name
DetectClassSquare: TypeAlias = dict[ClassInt, list[tuple[float, float, float, float]]]  # cls: [xc, yc, w, h]

YOLO8_MODEL_TYPE = Literal['yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x']

# ================ #
# Utility Function #
# ================ #

def copy_directory_contents(src_dir: Path | str, dst_dir: Path | str) -> None:
    """copy all contents from one the another directory"""
    src_path = Path(src_dir)
    dst_path = Path(dst_dir)
    dst_path.mkdir(parents=True, exist_ok=True)
    
    try:
        for item in src_path.iterdir():
            dst_item = dst_path / item.name
            if item.is_dir():
                shutil.copytree(item, dst_item, dirs_exist_ok=True)
            else:
                shutil.copy2(item, dst_item)
        fprint(f'All contents copied from {src_dir} to {dst_dir} successfully.')
    except Exception as e:
        fprint(f'Error: {e}', vtype='error')

        
def fprint(*msgs,
           vtype: Literal['info', 'io', 'warning', 'error', 'pass'] = 'info',
           timestamp: bool = True,
           **kwarg) -> None:
    """
    Formatting print with different colors based on verbose type

    :param msgs:
    :param vtype: verbose type
    :param timestamp: if print timestamp
    :return:
    """

    if vtype == 'error':
        prefix = '[ERROR]'
        color = 'red'
    elif vtype == 'warning':
        prefix = '[WARNING] '
        color = 'yellow'
    elif vtype == 'io':
        prefix = '[IO] '
        color = 'magenta'
    elif vtype == 'info':
        prefix = '[INFO]'
        color = 'cyan'
    elif vtype == 'pass':
        prefix = '[PASS]'
        color = 'green'
    else:
        raise ValueError(f'{vtype}')

    try:
        fg_color = getattr(Fore, color.upper())
    except AttributeError:
        fg_color = Fore.WHITE

    msg = fg_color + prefix
    if timestamp:
        msg += f"[{datetime.today().strftime('%y-%m-%d %H:%M:%S')}] - "

    try:
        out = f"{''.join(msgs)}\n"
    except TypeError:
        out = f'{msgs}'

    msg += out
    msg += Style.RESET_ALL
    print(msg, **kwarg)


def check_mps_available() -> bool:
    """for edge mac machine MPS backend check"""
    if not torch.backends.mps.is_available():
        if not torch.backends.mps.is_built():
            fprint('MPS not available because pytorch install not built with MPS enable', vtype='warning')
        else:
            fprint('MPS not available because current MacOs version is not 12.3+,'
                   ' or do not have MPS-enabled device on this machine', vtype='warning')
        return False
    else:
        fprint('MPS is available', vtype='pass')
        return True

# ============================= #
# Utility Function For Pipeline #
# ============================= #

def clone_png_dir(directory: Path | str,
                  resize_dim: tuple[int, int] | None = None) -> None:
    
    """Clone batch raw .npy files to png in a separated folder, image resize if needed

    :param directory: directory contains .npy files
    :param resize_dim: resize dim in (w, h) if not None
    """
    dst = Path(directory).parent / f'{directory.stem}_png'
    if not dst.exists():
        dst.mkdir()

    files = list(Path(directory).glob('*.npy'))
    iter_file = tqdm(files,
                     total=len(files),
                     unit='file',
                     desc=f'npy clone and resize as png to {dst}')

    for file in iter_file:
        img = np.load(file)
        if resize_dim is not None:
            img = cv2.resize(img, dsize=resize_dim, interpolation=cv2.INTER_NEAREST)

        out = (dst / file.name).with_suffix('.png')
        plt.imsave(out, img)

def detect_segmented_objects(seg_arr: np.ndarray, min_area: int = 500) -> DetectClassSquare:
    """
    Detects objects in a segmented image array and returns their center, width, and height.

    :param seg_arr: array of the segmented image where different values represent different objects
    :param min_area: minimum area threshold to consider an object. smaller areas are ignored
    :return: A dictionary where keys are object classes and values are lists of tuples containing
    (x_center, y_center, width, height) for each object of that class.
    """

    objects_info = {}

    # the unique values in the image array represent different classes
    object_classes = np.unique(seg_arr)
    # exclude the background class (value equal to 0)
    object_classes = object_classes[object_classes != 0]

    for object_class in object_classes:
        class_mask = (seg_arr == object_class)

        # label connected regions of the mask
        label_img = label(class_mask)
        regions = regionprops(label_img)

        class_objects_info = []
        for region in regions:

            if region.area < min_area:
                continue

            y0, x0, y1, x1 = region.bbox
            x_center = (x0 + x1) / 2
            y_center = (y0 + y1) / 2
            width = x1 - x0
            height = y1 - y0
            class_objects_info.append((x_center, y_center, width, height))

        objects_info[object_class - 1] = class_objects_info  # 1-base to 0-base

    return objects_info

        
def write_yolo_label_txt(img_filepath: str | Path,
                         detected: DetectClassSquare,
                         img_dim: tuple[int, int],
                         output_dir: str | Path) -> None:
    """
    Write detected object as yolov8 required label file for train dataset ::

        <class_id> <xc> <y_center> <width> <height>
    
    :param img_filepath: image filepath
    :param detected: `DetectClassSquare`
    :param img_dim: image dimension
    :param output_dir: txt output directory
    :return:
    """
    filename = Path(img_filepath).stem
    out = (Path(output_dir) / filename).with_suffix('.txt')

    width, height = img_dim
    with open(out, 'w') as file:
        for cls, info in detected.items():
            for (xc, yc, w, h) in info:
                x_center_normalized = xc / width
                y_center_normalized = yc / height
                width_normalized = w / width
                height_normalized = h / height
                content = f'{cls} {x_center_normalized} {y_center_normalized} {width_normalized} {height_normalized}\n'
                file.write(content)
                
                
def dir_ipy_imshow(directory: Path | str,
                   pattern: str = '*.png') -> None:
    """
    Display images from a directory with a button to load the next image

    :param directory: directory contain image sequences
    :param pattern: glob pattern in the directory
    :return:
    """
    from IPython.display import display
    from IPython.core.display import clear_output
    import ipywidgets as widgets

    files = sorted(list(Path(directory).glob(pattern)), key=lambda it: int(it.stem.split('_')[1]))
    iter_files = iter(files)

    image_display = widgets.Image()
    button = widgets.Button(description="Next Image")

    def on_button_clicked(b):
        try:
            file = next(iter_files)
        except StopIteration:
            clear_output(wait=True)
        else:
            with open(file, 'rb') as f:
                img = f.read()
            image_display.value = img

    button.on_click(on_button_clicked)
    display(button)
    display(image_display)
    on_button_clicked(None)

# ======================= #
# Folder Structure Handle #
# ======================= #


class ImageClsDir(NamedTuple):
    """Class for folder structure for train/test dataset and model/prediction output"""
    root_dir: Path

    @staticmethod
    def ensure_dir(p: Path):
        """auto mkdir, use for custom dir that fit for yolo pipeline"""
        p.mkdir(exist_ok=True)
        return p

    # ============= #
    # Train Dataset #
    # ============= #

    @property
    def train_data_dir(self) -> Path:
        return self.root_dir / 'train'

    @property
    def train_image_source(self) -> Path:
        return self.train_data_dir / 'img'

    @property
    def train_image_png(self) -> Path:
        return self.ensure_dir(self.train_data_dir / 'img_png')

    @property
    def train_seg_source(self) -> Path:
        return self.train_data_dir / 'seg'

    @property
    def train_seg_png(self) -> Path:
        return self.ensure_dir(self.train_data_dir / 'seg_png')

    @property
    def train_dataframe(self) -> pl.DataFrame:
        return pl.read_csv(self.train_data_dir / 'train_set.csv')

    # ============ #
    # Test Dataset #
    # ============ #

    @property
    def test_data_dir(self) -> Path:
        return self.root_dir / 'test'

    @property
    def test_image_source(self) -> Path:
        return self.test_data_dir / 'img'

    @property
    def test_image_png(self) -> Path:
        return self.ensure_dir(self.test_data_dir / 'img_png')

    # ================ #
    # Model Train/Eval #
    # ================ #

    @property
    def run_dir(self) -> Path:
        return self.root_dir / 'runs' / 'detect'

    def get_predict_dir(self, name: str) -> Path:
        return self.run_dir / name

    def get_predict_label_dir(self, name: str) -> Path:
        return self.get_predict_dir(name) / 'labels'

    def get_train_dir(self, name: str = 'train') -> Path:
        return self.run_dir / name

    def get_model_weights(self, name: str = 'train') -> Path:
        return self.run_dir / name / 'weights'

    
# =========================================== #
# Main Pipeline for YOLO classification model #
# =========================================== #

@final
class YoloUltralyticsPipeline:
    """Custom pipeline for implementing the YOLOv8 from ultralytics

    .. seealso :: `<https://docs.ultralytics.com/modes/train/>`_

    """

    def __init__(self,
                 root_dir: str | Path, *,
                 model_type: YOLO8_MODEL_TYPE = 'yolov8n',
                 model_path: str | Path | None = None,
                 resize_dim: tuple[int, int] | None = None,
                 use_gpu: bool = False,
                 epochs: int = 10,
                 batch_size: int = 32):
        """

        :param root_dir: `SOURCE_ROOT_DIR`. aka. input data source 
        :param model_type: {'yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x'}
        :param model_path: model path (If already trained)
        :param resize_dim: (w,h) resize for image 
        :param use_gpu: whether use gpu for fine-tuned the model
        :param epochs: number of the training epoch
        :param batch_size: training bathc size
        """
        self.image_dir = ImageClsDir(root_dir)
        self.resize_dim = resize_dim

        # label_dict, order sensitive
        df = self.image_dir.train_dataframe.drop('Id')
        self.label_dict: Final[dict[ClassInt, ClassName]] = {
            i: df.columns[i]
            for i in range(df.shape[1])
        }

        # training/prediction parameters
        self.model_type = model_type
        self.model = model_path  # if already fine-tuned. If None, then auto-inferred
        self._epochs = epochs
        self._lr0 = 0.01
        self._batch = batch_size
        self._train_filename: str | None = None  # incremental folder name, assign foreach train
        self._predict_filename: str | None = None  # incremental folder name, assign foreach predict

        # resources
        self._device = None  # torch.device('cpu')?
        if use_gpu:
            if torch.cuda.is_available():
                fprint('Process using cuda GPU')
                self._device = torch.device('cuda')
            elif check_mps_available():  # use cpu mode or increase the batch size if NMS time issue
                fprint('Process using mps GPU')
                self._device = torch.device('mps')
                self._lr0 = 0.00025
            else:
                fprint('none acceleration backend found', vtype='warning')
        
    def run(self) -> None:
        """main for pipeline if run in one-go"""
        # if fine-tuned model already specified
        if self.model is None:
            self.clone_png_dir()
            self.gen_yaml()
            self.gen_label_txt(debug_mode=False)
            self.yolo_train(model_type=self.model_type)
            self.yolo_predict(self.predict_filename)
            self.create_predicted_csv()
        else:
            self.yolo_predict(self.model)
            self.create_predicted_csv()
    
    @property
    def n_test(self) -> int:
        return len(list(self.image_dir.test_image_png.glob('*.png')))

    @property
    def train_filename(self) -> str:
        if self._train_filename is None:
            raise RuntimeWarning('run model train first')
        return self._train_filename

    @property
    def predict_filename(self) -> str:
        if self._predict_filename is None:
            raise RuntimeWarning('run model predict first')
        return self._predict_filename

    @property
    def cur_train_dir(self) -> Path:
        """current train directory"""
        return self.image_dir.get_train_dir(self.train_filename)

    @property
    def cur_predict_dir(self) -> Path:
        return self.image_dir.get_predict_dir(self.predict_filename)

    @property
    def cur_predict_label_dir(self) -> Path:
        return self.image_dir.get_predict_label_dir(self.predict_filename)
    
    def clone_png_dir(self) -> None:
        """Clone batch raw .npy files to png in a separated folder, image resize if needed"""
        fprint('<STATE 1> -> clone png dir')

        clone_png_dir(self.image_dir.train_image_source, self.resize_dim)
        clone_png_dir(self.image_dir.train_seg_source, self.resize_dim)
        clone_png_dir(self.image_dir.test_image_source)  # no need resize for prediction
    
    def gen_yaml(self, output: Path | str | None = None, verbose: bool = False) -> None:
        """
        Generate the yaml for yolov8 config

        :param output: output filepath of the yaml file
        :param verbose: show output verbose
        :return:
        """
        fprint('<STATE 2> -> generate yaml file')

        if output is None:
            output = self.image_dir.root_dir / 'dataset.yml'

        dy = {
            'path': str(self.image_dir.root_dir),
            'train': str(self.image_dir.train_image_png),
            'val': str(self.image_dir.train_image_png),
            'test': str(self.image_dir.test_image_png),
            'names': self.label_dict
        }

        with open(output, 'w') as file:
            yaml.safe_dump(dy, file, sort_keys=False)

        if verbose:
            with open(output, 'rb') as file:
                config = yaml.safe_load(file)
                pprint(config)
                
    def gen_label_txt(self, debug_mode: bool = True, min_area: int = 500) -> None:
        """
        Detect the object edge from seg files and generate the yolov8 required label file for train dataset ::

        <class_id> <xc> <y_center> <width> <height>

        :param debug_mode: debug mode to see the train dataset segmentation result
        :param min_area: minimum area threshold to consider an object. smaller areas are ignored
        :return:
        """
        fprint('<STATE 3> -> auto annotate segmentation file and generate label txt')

        files = sorted(list(self.image_dir.train_seg_source.glob('*.npy')),
                       key=lambda it: int(it.stem.split('_')[1]))

        iter_seg = tqdm(files,
                        total=len(files),
                        unit='file',
                        ncols=80,
                        desc='detect edge')

        for seg in iter_seg:
            im = np.load(seg)

            if self.resize_dim is not None:
                im = cv2.resize(im, dsize=self.resize_dim, interpolation=cv2.INTER_NEAREST)

            detected = detect_segmented_objects(im, min_area=min_area)

            if debug_mode:
                fprint(f'IMAGE -> {seg.stem}')
                fprint(f'DETECTED RESULT -> {detected}')
                fig, ax = plt.subplots(1, 2)

                # query raw
                _, _, idx = seg.stem.partition('_')
                file = list(self.image_dir.train_image_png.glob(f'*_{idx}.png'))[0]
                raw = Image.open(str(file))
                ax[0].imshow(raw)

                ax[1].imshow(im)

                # draw
                colors = plt.cm.rainbow(np.linspace(0, 1, 20))
                legend_handles = []
                for cls, info in detected.items():
                    color = colors[cls % len(colors)]  # cycle through colors
                    for (xc, yc, width, height) in info:
                        rect = patches.Rectangle((xc - width / 2, yc - height / 2),
                                                 width, height,
                                                 linewidth=1,
                                                 edgecolor=color,
                                                 facecolor='none')
                        ax[0].add_patch(rect)

                    legend_patch = patches.Patch(color=color, label=self.label_dict[cls])
                    legend_handles.append(legend_patch)

                ax[0].legend(handles=legend_handles, loc='best')

                plt.show()

            #
            write_yolo_label_txt(seg, detected,
                                 self.resize_dim if self.resize_dim is not None else im.shape,
                                 output_dir=self.image_dir.train_image_png)

    def yolo_train(self, model_type: YOLO8_MODEL_TYPE = 'yolov8n',
                   save: bool = True) -> None:
        """Load a pretrained model for training"""
        fprint(f'<STATE 4> -> Train the dataset using {model_type}')
        model_path = (self.image_dir.run_dir / model_type).with_suffix('.pt')
        model = YOLO(model_path)

        model.train(data=self.image_dir.root_dir / 'dataset.yml',
                    device=self._device,
                    batch=self._batch,
                    lr0=self._lr0,
                    # project=self.image_dir.run_dir, # bug from wandb dependency
                    epochs=self._epochs,
                    cache=True)

        self._train_filename = model.overrides['name']

        if save:
            model.export(format='onnx')

    def yolo_predict(self, model_path: Path | str | None = None,
                     save_plot: bool = True,
                     save_txt: bool = True):
        """
        Do the model prediction
        
        :param model_path: If None, use directly the last train model. Otherwise, specify the model path directly 
        :param save_plot: Save predict plot
        :param save_txt: Save the predict txt (DetectClassSquare for each image)
        :return: 
        """
        fprint('<STATE 5> -> Predicted result using test dataset')

        if model_path is None:
            model_path = self.image_dir.get_model_weights(self._train_filename) / 'best.pt'

        model = YOLO(model_path)
        model.predict(source=self.image_dir.test_image_png,
                      save=save_plot,
                      save_txt=save_txt,
                      project=self.image_dir.run_dir)

        self._predict_filename = model.predictor.save_dir.name
                   
    def create_predicted_csv(self, verbose: bool = True) -> pl.DataFrame:
        """create output csv"""
        fprint('<STATE 6> -> Write predicted result to csv')

        ret = {}
        for txt in self.cur_predict_label_dir.glob('test*.txt'):
            classes = set()
            with open(txt, 'r') as file:
                for line in file:
                    cls = line.split(' ')[0]
                    classes.add(cls)

            _, _, idx = txt.stem.partition('_')
            ret[idx] = list(classes)

        ret = dict(sorted(ret.items()))

        #
        dy = dict(Id=np.arange(self.n_test))
        for i, field in enumerate(self.label_dict.values()):
            dy[field] = np.full(self.n_test, 0)

        df = pl.DataFrame(dy)

        for i, classes in ret.items():
            for cls in classes:
                df[int(i), self.label_dict[int(cls)]] = 1

        dst = self.cur_predict_dir / 'test_set.csv'
        df.write_csv(dst)
        fprint(f'Successful create result in {dst}', vtype='io')
        
        if verbose:
            print(df)
        
        return df

    # ============================================== #
    # Visualization of Intermediate Training Metrics #
    # ============================================== #

    @classmethod
    def _fig_show(cls, p, size):
        fig = Image.open(p)
        plt.figure(figsize=size)
        plt.imshow(fig)
        plt.axis('off')

    @classmethod
    def show_confusion_matrix(cls, train_dir: Path | str):
        p = Path(train_dir) / 'confusion_matrix_normalized.png'
        cls._fig_show(p, (10, 10))

    @classmethod
    def show_epochs_progress(cls, train_dir: Path | str):
        p = Path(train_dir) / 'results.png'
        cls._fig_show(p, (10, 5))

    @classmethod
    def get_epochs_progress_dataframe(cls, train_dir: Path | str) -> pl.DataFrame:
        p = Path(train_dir) / 'results.csv'
        df = pd.read_csv(p)  # casting purpose
        return pl.from_pandas(df)


## Run the customize YOLOv8 classification pipeline step by step

In [ ]:
# To fit our pipeline, Copy dataset to kaggle `output` since `input` dir in READ-only
copy_directory_contents(src_dir='/kaggle/input/kul-h02a5a-computer-vision-ga2-2024', dst_dir='/kaggle/working/')

In [ ]:
# init our customized pipeline (enable CUDA kernel for accerlation)
yolo = YoloUltralyticsPipeline(root_dir=Path('/kaggle/working/'), 
                               resize_dim=(500, 500), 
                               model_type='yolov8n',
                               use_gpu=True,
                               epochs=100)

## **Step 1** - Clone batch raw .npy files to png in a separated folder

In [ ]:
yolo.clone_png_dir()

## **Step 2** - Generate the yaml for yolov8 config

In [ ]:
yolo.gen_yaml(verbose=True)

## **Step 3** - Detect the object edge from seg files and generate the yolo8 required label file for train dataset
- Each object example: \<class_id\> \<xc\> \<y_center\> \<width\> \<height\>

In [ ]:
# Use debug mode=True to see the detected examples 
yolo.gen_label_txt(debug_mode=False)

## **Step 4/5** - Load a pretrained model for training & prediction

In [ ]:
# Option 1: If not yet fine-tuned the model. 
yolo.yolo_train()
yolo.yolo_predict()

### See the intermediate figures from training/validation
### 1. Metrics throughout training/validation epochs

- `box_loss`: **bounding box regression loss during training**. It measures how well the predicted bounding boxes align with the ground truth bounding boxes. A decreasing trend indicates that the model is improving its bounding box predictions as training progresses.
  
- `cls_loss`: **classification loss during training**. It measures how well the model is classifying the objects within the bounding boxes. A decreasing trend indicates that the model is getting better at classifying objects correctly.
  
- `dfl_loss`: **distribution focal loss during training**. This loss typically focuses on the quality of the bounding box prediction, enhancing the performance on difficult examples. A decreasing trend indicates improving performance.

- `precision(B)`: **precision metric for the model on the training dataset**. Precision measures the proportion of true positive detections out of all positive detections (true positives + false positives). An increasing trend indicates the model is making fewer false positive errors.

- `recall(B)`:  **recall metric for the model on the training dataset**. Recall measures the proportion of true positive detections out of all actual positives (true positives + false negatives). An increasing trend indicates the model is detecting more true positives.

In [ ]:
train_dir = yolo.cur_train_dir
print(YoloUltralyticsPipeline.get_epochs_progress_dataframe(train_dir))
YoloUltralyticsPipeline.show_epochs_progress(train_dir)

### 2. Confusion matrix
 - During the model validation procedure, represent the normalized count of predictions for each class (0-1 in colormap)

In [ ]:
YoloUltralyticsPipeline.show_confusion_matrix(train_dir)

In [ ]:
# Option 2: If a fine-tuned model already exists. directly load
model_path =  ... # e.g., '/kaggle/working/runs/detect/train/weights/best.pt'
yolo.yolo_predict(model_path)

In [ ]:
# Brief interactive way to examine the predicted results using test dataset
predict_dir = yolo.image_dir.get_predict_dir(yolo.predict_filename)
dir_ipy_imshow(predict_dir, pattern='*.png')

## **Step 6** - Create csv for classified test dataset

In [ ]:
df = yolo.create_predicted_csv(verbose=True)

# 4. Adversarial attack
For this part, your goal is to fool your classification and/or segmentation CNN, using an *adversarial attack*. More specifically, the goal is build a CNN to perturb test images in a way that (i) they look unperturbed to humans; but (ii) the CNN classifies/segments these images in line with the perturbations.